<a href="https://colab.research.google.com/github/thesteuber/assign03-50-steuber/blob/2-read-in-hr-data-to-new-notebook/churn_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Machine Learning - Assignment 3 "Employee Churn" 


##Purpose
Gain experience applying scikit learn to machine learning problems.

##Import Python Libraries
Run the following python to initiliaze all the needed libraries for the notebook.

In [1]:
#tables and visualizations
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

#machine learning
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline 
from sklearn.compose import ColumnTransformer, make_column_selector
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, LabelBinarizer, StandardScaler
from sklearn import config_context
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay

##Tasks

###1. Create a new notebook 


---


Create a new notebook named "churn-prediction", and properly document the notebook with descriptions of the tasks.

Task completed as seen in this notebook.

###2. Read in HR Data


---


Using the scikit-demo-solutions notebook as a guide, read in the HR data.

###3. Split the Data


---


Split the data for train and test.

###4. Build Pipeline


---


Build an appropriate training pipeline.

###5. Cross Validation


---


Carry out cross-validation